In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import collections
import json
import matplotlib.cm as cm
import numpy as np 
from scipy import signal
import math

import dash
from dash import dcc
from dash import html

# Theming

Define default plot layout

In [2]:
layout_settings = {
    'paper_bgcolor':'rgba(0,0,0,0)',
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'title_font_family': 'Open Sans',
    'title_font_color': 'black',
    'font': {
        'family': 'Open Sans',
        'color': 'black'
    }
}
transparent_layout = go.Layout(layout_settings)

Select color theme

In [3]:
cmap_primary = cm.get_cmap('Greens')
cmap_secondary = cm.get_cmap('Reds')
# See: https://matplotlib.org/stable/tutorials/colors/colormaps.html

def to_rgba(percentage, cmap):
    return f'rgba{str(cmap(percentage))}'

primary_colors = [to_rgba(n, cmap_primary) for n in np.linspace(0.1, 1, 5)]
secondary_colors = [to_rgba(n, cmap_secondary) for n in np.linspace(0.1, 1, 5)]
primary_colors, secondary_colors

(['rgba(0.9132641291810842, 0.9667051134179162, 0.8961937716262975, 1.0)',
  'rgba(0.6903960015378701, 0.8758323721645521, 0.6653133410226836, 1.0)',
  'rgba(0.3764705882352941, 0.7301806997308727, 0.42429834678969625, 1.0)',
  'rgba(0.10818915801614765, 0.5201845444059976, 0.24982698961937716, 1.0)',
  'rgba(0.0, 0.26666666666666666, 0.10588235294117647, 1.0)'],
 ['rgba(0.9969242599000384, 0.8961937716262975, 0.8489042675893886, 1.0)',
  'rgba(0.9882352941176471, 0.6362322183775471, 0.5200615148019992, 1.0)',
  'rgba(0.9658592848904267, 0.34340638216070746, 0.24405997693194925, 1.0)',
  'rgba(0.7645213379469434, 0.08664359861591696, 0.10708189158016147, 1.0)',
  'rgba(0.403921568627451, 0.0, 0.05098039215686274, 1.0)'])

# Import / prepare data

## Map the rgb codes to readable names 

In [4]:
with open('data/rgb-name-map.json', 'r') as file:
    rgb_name_map = json.load(file)
rgb_name_map

{'rgb(166,230,116)': 'FOPH media release',
 'rgb(249,127,24)': 'Major events prohibited',
 'rgb(170,67,105)': 'Entry restrictions',
 'rgb(255,91,91)': 'Closure of stores, schools, etc',
 'rgb(164,0,0)': 'Maximum number of persons at meetings and in buildings',
 'rgb(237,153,0)': 'Existing measure extended',
 'rgb(164,255,164)': 'Relaxation of measures',
 'rgb(255,255,80)': 'Test Measures',
 'rgb(40,40,40)': 'Mask obligation',
 'rgb(48,48,225)': 'Vaccinations',
 'rgb(183,183,183)': 'Home office requirement or recommendation',
 'rgb(112,72,0)': 'Restrictions with certificates',
 'rgb(255,128,255)': 'Certificate',
 'rgb(0,0,0)': 'Unclassified'}

## Import extraordinary events

In [5]:
with open('data/extraordinary-events.json', 'r') as file:
    extraordinary_events = json.load(file)
extraordinary_events

[['2020-03-16', 'Extraordinary situation'],
 ['2020-06-19', 'Restrictions lifted'],
 ['2020-10-28', 'Mask requirement'],
 ['2021-01-04', 'Beginning vaccination'],
 ['2021-10-20', '1. Booster vaccination'],
 ['2021-11-03', 'Enhanced measures']]

## Importing the OWID Dataset

In [6]:
df = pd.read_csv("data/owid-covid-data.csv", low_memory=False)
df = df.query("location == 'Switzerland'")
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df = df.reset_index(drop=True)


In [7]:
df['cases_over_time'] = np.cumsum(df['new_cases'].fillna(0))
df['deaths_over_time'] = np.cumsum(df['new_deaths'].fillna(0))

In [8]:
df['mortality_rate'] = df['deaths_over_time'] / df['cases_over_time'] * 100
df['mortality_rate'].max(), df['mortality_rate'].min()

(5.835716142448661, 0.0)

In [9]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,cases_over_time,deaths_over_time,mortality_rate
0,CHE,Europe,Switzerland,2020-02-25,1.0,1.0,NaN,NaN,NaN,NaN,...,83.78,0.955,8740471.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0
1,CHE,Europe,Switzerland,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,83.78,0.955,8740471.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0
2,CHE,Europe,Switzerland,2020-02-27,8.0,7.0,NaN,NaN,NaN,NaN,...,83.78,0.955,8740471.0,NaN,NaN,NaN,NaN,8.0,0.0,0.0
3,CHE,Europe,Switzerland,2020-02-28,8.0,0.0,NaN,NaN,NaN,NaN,...,83.78,0.955,8740471.0,NaN,NaN,NaN,NaN,8.0,0.0,0.0
4,CHE,Europe,Switzerland,2020-02-29,18.0,10.0,NaN,NaN,NaN,NaN,...,83.78,0.955,8740471.0,NaN,NaN,NaN,NaN,18.0,0.0,0.0


Creating a dict for all annotations, so annotations can be added at a specific date with the matching covid cases at this day.

In [10]:
#Date, Number of restriction, color, category
with open('data/wikipedia_restrictions.json', 'r') as file:
    restrictions_annotations = json.load(file)
restrictions_annotations_date_format = '%Y-%m-%d'
len(restrictions_annotations)

62

## Prepare Wave data

JSON file that defines the different covid wave durations

In [11]:
#Date, Number of restriction, color, category
with open('data/covid-waves.json', 'r') as file:
    waves_dict = json.load(file)
wave_date_format = '%d-%m-%Y'
df_waves = pd.DataFrame.from_dict(waves_dict, orient="index", columns=['start', 'end'])
df_waves['start'] = pd.to_datetime(df_waves['start'], format=wave_date_format)
df_waves['end'] = pd.to_datetime(df_waves['end'], format=wave_date_format)

Calculate how many new cases in each wave

In [12]:
wave_new_cases = []
for index, row in df_waves.iterrows():
    start = row['start']
    end = row['end']
    mask = (df['date'] >= start) & (df['date'] <= end)
    wave_df = df.loc[mask]
    total_cases = wave_df['new_cases'].sum()
    wave_new_cases.append(total_cases)
df_waves['new_cases'] = wave_new_cases

Scale cases between 50 and 100 for bubble size

In [13]:
values = df_waves['new_cases']

# Get the minimum and maximum values
min_value = min(values)
max_value = max(values)

# Scale the values
scaled_values = [(value - min_value) / (max_value - min_value) * (100 - 25) + 25 for value in values]

df_waves['scaled_cases_blob'] = scaled_values
values = None

df_waves.head()

,start,end,new_cases,scaled_cases_blob
First wave,2020-02-27,2020-06-19,31234.0,25.000000
Second wave,2020-10-06,2021-06-23,646346.0,49.783902
Third wave,2021-08-09,2021-10-19,134230.0,29.149883
Fourth wave,2021-10-30,2022-02-26,1892660.0,100.000000
Fifth wave,2022-02-27,2022-04-10,763471.0,54.503066


In [14]:
restrictions_annotations_dates = [
    datetime.strptime(v[0], restrictions_annotations_date_format) for v in restrictions_annotations]

def count_waves(row):
    count = len([v_r for v_r in restrictions_annotations_dates if row['start'] <= v_r <= row['end']])
    return count

df_waves["wave_counts"] = df_waves.apply(count_waves, axis=1)
df_waves.head()

,start,end,new_cases,scaled_cases_blob,wave_counts
First wave,2020-02-27,2020-06-19,31234.0,25.000000,19
Second wave,2020-10-06,2021-06-23,646346.0,49.783902,18
Third wave,2021-08-09,2021-10-19,134230.0,29.149883,4
Fourth wave,2021-10-30,2022-02-26,1892660.0,100.000000,13
Fifth wave,2022-02-27,2022-04-10,763471.0,54.503066,2


# Graph: Cases and extraordinary events

In [15]:
# Drop empty so they connect instead of having NA values
df_new_cases = df[["date", "new_cases_smoothed", "new_cases"]].dropna()
df_new_deaths = df[["date", "new_deaths_smoothed", "new_deaths"]].dropna()

# Create figure with secondary y-axis
fig_cases_extraordinary_events = make_subplots(specs=[[{"secondary_y": True}]])

fig_cases_extraordinary_events.add_trace(
    go.Scatter(x=list(df_new_cases.date), y=list(df_new_cases.new_cases_smoothed.astype(int)), name="New cases smoothed", 
               text=df_new_cases.new_cases.astype(int),
               mode="lines", marker_color=primary_colors[-1], hoverlabel = dict(namelength = -1),
               hovertemplate='New cases smoothed: %{y: ,d} (absolute: %{text: ,d})<extra></extra>'), 
    secondary_y=False)

fig_cases_extraordinary_events.add_trace(
    go.Scatter(x=list(df_new_deaths.date), y=list(df_new_deaths.new_deaths_smoothed.astype(int)), name="New deaths smoothed", 
               text=df_new_deaths.new_deaths.astype(int),
               mode="lines", marker_color=primary_colors[-2], hoverlabel = dict(namelength = -1),
               hovertemplate='New deaths smoothed: %{y: ,d} (absolute: %{text: ,d})<extra></extra>'),
    secondary_y=True)

fig_cases_extraordinary_events.update_layout(
    xaxis=dict(
        gridcolor='rgba(0, 0, 0, 0)',
        gridwidth=0
    ),
    yaxis=dict(
        gridcolor='rgba(0, 0, 0, 0)',
        gridwidth=0
    )
)

# Add vertical lines and text annotations for extraordinary_events
fig_cases_extraordinary_events.update_layout(**layout_settings,
    xaxis=dict(
        # Add range selector
        rangeslider=dict(
            visible=True
        )
    ),
    # Add vertical lines and text annotations for extraordinary_events
    shapes=[
        dict(
            type="line",
            x0=x,
            y0=0,
            x1=x,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color=secondary_colors[-1], width=3),
        ) for x, _ in extraordinary_events
    ],
    annotations=[
        dict(
            x=x,
            y=1,  # Display annotations at the top of the plot
            text=text,
            showarrow=False,
            yshift=20 * (index % 2 + 1),  # Shift annotations upwards by 20 pixels
            xshift=0,  # Shift annotations to the right by 20 pixels
            textangle=0,  # Display text vertically
            xref="x",
            yref="paper",  # Display annotations outside of the graph
        ) for index, (x, text) in enumerate(extraordinary_events)
    ],
    hovermode='x unified'
)

fig_cases_extraordinary_events.show()

# Graph: Number of restrictions / wave

In [16]:
fig_num_restrictions_wave = go.Figure(layout=transparent_layout)

color_map = {
    '15+': primary_colors[-1],
    '10-14': primary_colors[-2],
    '0-9': primary_colors[-3]
}
legend_desc = ['15+', '10-14', '0-9']

# Keep track of the legend groups that have been added
legend_groups_added = set()

for index, row in df_waves.iterrows():
    v = row['wave_counts']
    legend_desc_selected = legend_desc[0] if v > 14 else (legend_desc[1] if v > 9 else legend_desc[2])
    
    # Set showlegend to True for the first entry of each legend group and False for all subsequent entries
    showlegend = legend_desc_selected not in legend_groups_added
    legend_groups_added.add(legend_desc_selected)
    hover_text = f"""From {row['start'].strftime('%b %d, %Y')} until {row['end'].strftime('%b %d, %Y')}
    <br>New cases: {row['new_cases']:,.0f}
    """
    
    fig_num_restrictions_wave.add_trace(go.Scatter(
        x=(index,), y=(v, ),
        name=legend_desc_selected,
        legendgroup=legend_desc_selected,
        mode='markers',
        text=(hover_text, ),
        hovertemplate='%{text}<br>Restrictions: %{y}<extra></extra>',
        hoverlabel={'font': {'color': 'white'}, 'bordercolor': 'white'},
        marker_size=row['scaled_cases_blob'],
        marker_color=color_map[legend_desc_selected],
        showlegend=showlegend
    ))

fig_num_restrictions_wave.update_layout(
    title='Number of restrictions per wave',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of restrictions',
        titlefont_size=16,
        tickfont_size=14,
        fixedrange=True
    ),
    xaxis=dict(
        fixedrange=True
    ),
    showlegend=True
)

fig_num_restrictions_wave.show()

# Graph: Stringency Index in Switzerland

In [17]:
df_stringency = df[['date', 'stringency_index']].copy().dropna()
df_mortality_rate = df[["date", "mortality_rate"]].dropna()

fig_stringency_index = make_subplots(specs=[[{"secondary_y": True}]])


fig_stringency_index.add_trace(
    go.Scatter(
        x=df_mortality_rate.date, 
        y=signal.savgol_filter(df_mortality_rate.mortality_rate, 51, 3), 
        text=df_mortality_rate.mortality_rate,
        name="Mortality rate", 
        mode="lines", 
        hoverlabel = dict(namelength = -1),
        hovertemplate='Mortality rate: %{text:.2f}%<extra></extra>',
        fill='tozeroy',
        line={'color': secondary_colors[-2]},
    ), secondary_y=False
)

fig_stringency_index.add_trace(
    go.Scatter(
        name='Stringency index',
        x=df_stringency['date'],
        y=df_stringency['stringency_index'], 
        mode='lines', 
        line={'color': primary_colors[-1]},
        hovertemplate='Stringency index: %{y}<extra></extra>',
        showlegend=True
    ), secondary_y=True
)

fig_stringency_index.update_layout(**layout_settings,
    hovermode='x unified',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Stringency points',
        titlefont_size=16,
        tickfont_size=14,
        side='right'
    ),
    yaxis2=dict(
        side='left'
    ),
    xaxis=dict(
        # Add range selector
        rangeslider=dict(
            visible=True
        )
    ),
    showlegend=True,
    # Add vertical lines and text annotations for extraordinary_events
    shapes=[
        dict(
            type="line",
            x0=x,
            y0=0,
            x1=x,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color=primary_colors[3], width=3),
        ) for x, _ in extraordinary_events
    ],
    annotations=[
        dict(
            x=x,
            y=1,  # Display annotations at the top of the plot
            text=text,
            showarrow=False,
            yshift=20 * (index % 2 + 1),  # Shift annotations upwards by 20 pixels
            xshift=0,  # Shift annotations to the right by 20 pixels
            textangle=0,  # Display text vertically
            xref="x",
            yref="paper",  # Display annotations outside of the graph
        ) for index, (x, text) in enumerate(extraordinary_events)
    ])

fig_stringency_index.show()

# Graph: Restriction frequency

In [18]:
#count all colors in dict which appear in the dict
def count_colors_in_dict(determine_frequency):
    sorted_counted_colors = collections.Counter([v[1] for v in [de_freq[1] for de_freq in determine_frequency]]).most_common()
    return {v[0]: [v[1], rgb_name_map[v[0]]] for v in sorted_counted_colors}

categories_dict = count_colors_in_dict(restrictions_annotations)
categories_dict

{'rgb(166,230,116)': [11, 'FOPH media release'],
 'rgb(164,255,164)': [10, 'Relaxation of measures'],
 'rgb(170,67,105)': [9, 'Entry restrictions'],
 'rgb(255,91,91)': [5, 'Closure of stores, schools, etc'],
 'rgb(112,72,0)': [5, 'Restrictions with certificates'],
 'rgb(164,0,0)': [4, 'Maximum number of persons at meetings and in buildings'],
 'rgb(237,153,0)': [4, 'Existing measure extended'],
 'rgb(255,255,80)': [3, 'Test Measures'],
 'rgb(255,128,255)': [3, 'Certificate'],
 'rgb(40,40,40)': [2, 'Mask obligation'],
 'rgb(48,48,225)': [2, 'Vaccinations'],
 'rgb(183,183,183)': [2, 'Home office requirement or recommendation'],
 'rgb(249,127,24)': [1, 'Major events prohibited'],
 'rgb(0,0,0)': [1, 'Unclassified']}

In [19]:
categories = [v[1] for v in categories_dict.values()]
counts = [v[0] for v in categories_dict.values()]
df_category_frequency = pd.DataFrame(data=categories, index=counts, columns=['category'])

In [20]:
fig_restriction_frequency = go.Figure(layout=transparent_layout)

color_map = {
    '10+': primary_colors[-1],
    '5-9': primary_colors[-2],
    '0-4': primary_colors[-3]
}
legend_desc = ['10+', '5-9', '0-4']

# Keep track of the legend groups that have been added
legend_groups_added = set()

for index, row in df_category_frequency.iterrows():
    legend_desc_selected = legend_desc[0] if index > 9 else (legend_desc[1] if index > 4 else legend_desc[2])

    # Set showlegend to True for the first entry of each legend group and False for all subsequent entries
    showlegend = legend_desc_selected not in legend_groups_added
    legend_groups_added.add(legend_desc_selected)

    fig_restriction_frequency.add_trace(go.Bar(
        x=(row['category'],), y=(index, ),
        name=legend_desc_selected,
        marker_color=color_map[legend_desc_selected],
        legendgroup=legend_desc_selected,
        hovertemplate='%{x}<br>Frequency: %{y}<extra></extra>',
        hoverlabel={'font': {'color': 'white'}, 'bordercolor': 'white'},
        showlegend=showlegend
    ))

fig_restriction_frequency.update_layout(
    title='Frequency of measures taken',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of restrictions',
        titlefont_size=16,
        tickfont_size=14,
    ), showlegend=True
)
fig_restriction_frequency.show()

# Graph: Restrictions time occurrence

In [21]:
df_annotations = pd.DataFrame(data=[de_freq[1] for de_freq in restrictions_annotations], 
                              index=[de_freq[0] for de_freq in restrictions_annotations],
                              columns=['ignore', 'color', 'restrictions', 'category', "test"])

df_annotations['category'] = df_annotations['color'].map(categories_dict)
df_annotations['category'] = df_annotations['category'].apply(lambda x: x[1])
df_annotations.drop('ignore', axis=1, inplace=True)
df_annotations.head()

,color,restrictions,category,test
2020-02-27,"rgb(166,230,116)",press-release,FOPH media release,None
2020-02-28,"rgb(249,127,24)",major-event-banned,Major events prohibited,None
2020-03-11,"rgb(170,67,105)",Entry restrictions,Entry restrictions,None
2020-03-13,"rgb(255,91,91)",closure of schools,"Closure of stores, schools, etc",limit number of people
2020-03-16,"rgb(166,230,116)",extraordinary situation,FOPH media release,None


In [22]:
# Get the counts of each category and sort
counts = df_annotations['category'].value_counts().sort_values(ascending=False)

# Map the counts to the original dataframe
df_annotations['category_occurrences'] = df_annotations['category'].map(counts)

# Sort the dataframe by the count column
df_annotations.sort_values(by='category_occurrences', ascending=False, inplace=True)

df_annotations.head()

,color,restrictions,category,test,category_occurrences
2020-02-27,"rgb(166,230,116)",press-release,FOPH media release,None,11
2022-03-30,"rgb(166,230,116)",Change to normal situation,FOPH media release,None,11
2022-02-16,"rgb(166,230,116)",Removal of almost all restrictions,FOPH media release,None,11
2020-03-16,"rgb(166,230,116)",extraordinary situation,FOPH media release,None,11
2022-02-04,"rgb(166,230,116)",Removal of PCR testing coverage,FOPH media release,None,11


In [23]:
fig_restrictions_time_occurrence = go.Figure(layout=transparent_layout)

color_map = {
    '10+': primary_colors[-1],
    '5-9': primary_colors[-2],
    '0-4': primary_colors[-3]
}
legend_desc = ['10+', '5-9', '0-4']

# Keep track of the legend groups that have been added
legend_groups_added = set()

for index, row in df_annotations.iterrows():
    v = row['category_occurrences']
    legend_desc_selected = legend_desc[0] if v > 9 else (legend_desc[1] if v > 4 else legend_desc[2])

    # Set showlegend to True for the first entry of each legend group and False for all subsequent entries
    showlegend = legend_desc_selected not in legend_groups_added
    legend_groups_added.add(legend_desc_selected)

    fig_restrictions_time_occurrence.add_trace(go.Scatter(
        x=(row['category'],), y=(index, ),
        name=legend_desc_selected,
        legendgroup=legend_desc_selected,
        mode='markers',
        text=(row['category_occurrences'], ),
        hovertemplate='%{y}, %{x}<br>Total occurrences: %{text}<extra></extra>',
        hoverlabel={'font': {'color': 'white'}, 'bordercolor': 'white'},
        marker_size=10,
        marker_color=color_map[legend_desc_selected],
        showlegend=showlegend
    ))

fig_restrictions_time_occurrence.update_layout(title="Categorical COVID-19 restrictions over time", 
                   showlegend=True,
                   yaxis=dict(
                       fixedrange=True
                   ))

fig_restrictions_time_occurrence.show()

In [24]:
# Add the Google font
external_stylesheets = ['https://fonts.googleapis.com/css2?family=Open+Sans&display=swap', 'style.css']
    
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Impact COVID-19 Measures in Switzerland'
app._favicon = "favicon.png"

app.layout = html.Div(children=[
    html.H1('Impact COVID-19 Measures in Switzerland'),
    html.P([
        html.Span('By '),
        html.B('Joël A. Kessler, Ramon Stoffel'),
        html.Span(' and '),
        html.B('Tim Büsching')
    ], style={'textAlign': 'center'}),
    html.P('Published Mon Jan 09 2022', className='all-caps', style={'textAlign': 'center', 'paddingBottom': '50px'}),
    html.Div([
        html.Div([
            html.Div([
                html.H2('Introduction and Motivation'),
                html.P(
                    '''During the Covid-19 pandemic, the government imposed numerous measures. As a citizen, it was often difficult to understand which measure had what impact. 
                    Now that the pandemic in Switzerland is over, we would like to give the reader or the citizen a better overview with the available data. 
                    In addition, we try to show when which measure was used. ''')
            ]),
            html.Div([
                html.H2('Providing an Overview of the Measures'),
                html.P(
                    '''In this section, we provide an overview of the individual measures implemented during the Covid-19 pandemic in Switzerland. 
                    Initially, we planned to display all measures in a single plot. However, we realized that this approach would not be effective as 
                    it would present too much information in a small time frame. Therefore, we plotted the case and death counts from the OWID dataset 
                    to provide context for the measures. We have selected only the most significant measures to be included in our first chart, 
                    allowing for a general understanding of the impact of each measure on the progression of the pandemic. The case and death figures can be 
                    toggled on or off by clicking on the legend. In subsequent charts, we will delve deeper into the specific measures and their effects.'''),
                dcc.Graph(figure=fig_cases_extraordinary_events)
            ]),
            html.Div([
                html.H2('Number of Measures Implemented During Each Wave of the Covid-19 Pandemic'),
                html.P(
                    '''To further understand the trends in measures taken, we examined the number of measures implemented per wave of the pandemic. 
                    It is evident that the highest number of measures were implemented during the first two waves, as the novel Covid-19 virus was still largely 
                    unknown and there was a need for swift action to mitigate its spread. The fourth wave also saw a significant number of measures, 
                    likely due to the emergence of the Omikron variant, which, while not as severe as previous strains, had the ability to rapidly spread and 
                    contributed to a spike in cases. Later in our analysis, we will delve into the specific measures that were 
                    most frequently implemented during each wave. For each infection wave, we also visualized the number of cases through the size of the points. 
                    This allows you to clearly see, for example, that although we had very low case numbers in the first wave, 
                    the number of measures was higher than in the fourth wave when we reached the peak of cases. The number of cases are displayed by hovering over them.'''),
                dcc.Graph(figure=fig_num_restrictions_wave)
                ]),
            html.Div([
                html.H2('COVID-19 Stringency Index: A Comparison of Strictness of Government Policies for Vaccinated, Non-Vaccinated, and National Average Populations'),
                html.P(
                    '''The Stringency Index is a tool created by the University of Oxford that aims to measure the strictness of government 
                    policies related to the COVID-19 pandemic. "It is calculated using nine metrics, school closures; workplace closures; cancellation of public events; 
                    restrictions on public gatherings; closures of public transport; stay-at-home requirements; public information campaigns; 
                    restrictions on internal movements; and international travel controls, with a higher score indicating a stricter response. 
                    The index is calculated for three categories: vaccinated individuals, non-vaccinated individuals, 
                    and a national average that is weighted based on vaccination rates.'''),
                html.P(
                    '''It is important to note that the Stringency Index simply measures the 
                    strictness of government policies and does not imply the appropriateness or effectiveness of a particular country's response."'''),
                html.P([
                    html.Span('Citation: '),
                    html.A('OWID Stringency Index explanation', href='https://ourworldindata.org/covid-stringency-index')
                ]),
                html.P('''
                    In comparison with our analysis of the number of measures, we found similarities between the two, with both showing increased 
                    levels of strictness during the first and second waves of the pandemic and during the fourth wave. 
                    To relate the Stringency Index to another metric, we calculated the mortality rate and added it to the chart. 
                    The mortality rate for the COVID-19 outbreak in 2020 should be taken with caution, as at the beginning, 
                    not many tests were conducted. This can lead to an overestimation of the mortality rate without it actually being as high. 
                    We calculate the mortality rate by dividing the rolling number of deaths by the number of new cases.'''),
                dcc.Graph(figure=fig_stringency_index)
            ]),
            html.Div([
                html.H2('Frequency of Measures Taken in Switzerland in Response to COVID-19'),
                html.P([
                    html.Span('In a '),
                    html.A('Wikipedia article', href='https://de.m.wikipedia.org/wiki/Chronologie_der_Reaktionen_und_Massnahmen_infolge_der_COVID-19-Pandemie_in_der_Schweiz'),
                    html.Span(
                        ''', all measures taken in Switzerland were listed chronologically. 
                        From this article, we then created a dictionary in which we categorized the restrictions. 
                        The following chart shows the frequency of the individual categories.''')
                ]),
                html.P(
                    '''The most frequent measures such as media release, relaxtion of measures as well as the entry restrictions 
                    are particularly common, as these were more or less the only agents that could be used at the beginning of the pandemic. Restrictions on use were tightened as new virus variants emerged. 
                    First, citizens had to be informed frequently about the current situation. 
                    The virus and its effects were still very much unknown. '''),
                html.P(
                    '''After the first wave, the measures taken were quickly lifted again, as the number of cases almost returned 
                    to a zero level in the summer. All subsequent measures were also lifted, 
                    which is why this category is also frequently represented. '''),
                html.P('''Entry restrictions were used frequently, especially in the first waves, 
                because the government needed to prevent entrants from infecting the population.'''),
                html.P(
                    '''Measures such as mask obligation or test measures occurred much less frequently, contrary to our expectations. 
                    We attribute this to the fact that these measures were in place for a very long time and proved to be effective. 
                    In the case of testing, only the cost coverage was changed, but these measures then fall into the category of media releases.'''),
                dcc.Graph(figure=fig_restriction_frequency)
            ]),
            html.Div([
                html.H2('Illustrating the Implementation of Measures Over Time: A Sequence and Usage Analysis'),
                html.P(
                    '''In this chart, we illustrate the arrangement of the individual measures in relation to their implementation over time. 
                    By presenting the data in this way, we are able to clearly convey the sequence and usage of the measures, 
                    improving the overall understanding of the information.'''),
                html.P(
                    '''For each category, we can now clearly see when and how often it was used. 
                    Media relases were used for almost the entire duration of the pandemic. 
                    The chart also shows the temporal concentration of the measures. The entry restrictions, which were made at the beginning 
                    of the pandemic and then adjusted to the current situation at the end, or finally lifted completely, are a good example.'''),
                dcc.Graph(figure=fig_restrictions_time_occurrence)
            ]),
            html.Div([
                html.H2('Conclusion'),
                html.P(
                    '''Finally, we can say that with our data story, we were able to provide a good overview of the pandemic and its measures. 
                    During our work, we noticed that it is a balancing act for the government to take the right measure at the best time without annoying the citizen, but effectively protecting them from the virus. 
                    While working with the data, we also noticed how complex and diverse the topic is. 
                    With the visualizations, we learned how to simplify this complexity somewhat and present the information to the reader in a meaningful way.''')
            ]),
            html.Div([
                html.H3('Sources'),
                html.P([
                    html.A([
                        html.Span('Our World in Data, (2022), GitHub repository')
                    ], href='https://github.com/owid/covid-19-data/tree/master/public/data')
                ]),
                html.P([
                    html.A([
                        html.Span('Chronologie der Reaktionen und Massnahmen infolge der COVID-19-Pandemie in der Schweiz, (2022), Wikipedia')
                    ], href='https://de.m.wikipedia.org/wiki/Chronologie_der_Reaktionen_und_Massnahmen_infolge_der_COVID-19-Pandemie_in_der_Schweiz')
                ])
            ])
        ], className='wrapper')
    ])
])
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [09/Jan/2023 16:52:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2023 16:52:27] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2023 16:52:27] "GET /assets/style.css?m=1673274801.0768213 HTTP/1.1" 304 -
127.0.0.1 - - [09/Jan/2023 16:52:27] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2023 16:52:28] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2023 16:52:28] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2023 16:52:28] "GET /assets/favicon.png?m=1673210760.2757347 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2023 16:52:28] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jan/2023 16:52:28] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
